In [4]:
from pdbfixer import PDBFixer
from pathlib import Path
import mdtraj as md
import openmm as mm 
from openmm import app
from openmm import unit as U
import nglview as view

In [1]:
from simulation.system_builder import SMOGSystem

    ****************************************************************************************    
      **** *** *** *** *** *** *** *** OpenSMOG (v1.1.1) *** *** *** *** *** *** *** ****       

           The OpenSMOG class is used to perform molecular dynamics simulations using           
                     Structure-Based Models (SBM) for biomolecular systems,                     
             and it allows for the simulation of a wide variety of potential forms.             
                      OpenSMOG uses force field files generated by SMOG 2.                      
                             OpenSMOG documentation is available at                             
                  https://opensmog.readthedocs.io and https://smog-server.org                   

                    OpenSMOG is described in: Oliveira and Contessoto et al,                    
              SMOG 2 and OpenSMOG: Extending the limits of structure-based models.              
                    Protein 

In [2]:
handler = SMOGSystem()

In [3]:
handler.build_simulation()

OpenMMException: CustomNonbondedForce must have exactly as many particles as the System it belongs to.

In [2]:
struc = PDBFixer("../data/2efv_fixed.pdb")
struc.findMissingResidues()
struc.findMissingAtoms()
struc.addMissingAtoms()
struc.addMissingHydrogens()
top = struc.topology
pos = struc.positions

model = app.Modeller(top, pos)

topology = model.getTopology()
box_vectors = topology.getPeriodicBoxVectors()

In [14]:
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')

model.addSolvent(
    forcefield,
    padding=1.*U.nanometer,
    model='tip3p',
    ionicStrength=0.1 * U.millimolar, 
    positiveIon='Na+',
    negativeIon='Cl-'
)

# Write model to the pdb file
with open('../data/prototype1/setup.pdb', 'wt') as f:
    app.PDBFile.writeFile(model.getTopology(), model.getPositions(), f)


system = forcefield.createSystem(
    model.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=1.0*U.nanometers,
    constraints=app.HBonds,
    ewaldErrorTolerance=0.0005
)

In [15]:
with open('../data/prototype1.xml', 'w') as f:
    system_xml = mm.XmlSerializer.serialize(system)
    f.write(system_xml) 

In [17]:
integrator = mm.openmm.LangevinIntegrator(
    300*U.kelvin, 
    1/U.picosecond, 
    0.002 * U.picoseconds
)
platform = mm.Platform.getPlatformByName("CPU")
simulation = app.simulation.Simulation(
    model.topology,
    system,
    integrator,
    platform
)

simulation.context.setPositions(model.positions)

In [18]:
simulation.reporters.append(app.pdbreporter.PDBReporter("../data/prototype1/run1.pdb", 50))
simulation.reporters.append(app.dcdreporter.DCDReporter('../data/prototype1/run1.dcd', 10))
simulation.reporters.append(app.statedatareporter.StateDataReporter('mm.log', 10, step=True, potentialEnergy=True, temperature=True))

In [19]:
simulation.minimizeEnergy()

In [20]:
### Get out structure after minimization (Bonus trick)
current_reporter = app.pdbreporter.PDBReporter("../data/prototype1/structure_after_minimization.pdb",1)
current_reporter.report(simulation, simulation.context.getState(getPositions=True))

In [21]:
minimized = md.load_pdb("../data/prototype1/structure_after_minimization.pdb")
view.show_mdtraj(minimized)

NGLWidget()

In [22]:
n_steps = 10000
simulation.step(n_steps)

In [3]:
traj1 = md.load_pdb("../data/prototype1/run1.pdb")
view.show_mdtraj(traj1)

NGLWidget(max_frame=199)

In [33]:
base = md.load_pdb("../data/2efv.pdb")
print(list(base.topology.chains))
view.show_mdtraj(base)


[<mdtraj.core.topology.Chain object at 0x15f4a5a90>, <mdtraj.core.topology.Chain object at 0x15f4a5af0>]


NGLWidget()

In [3]:
base = md.load_pdb("../data/2efv_fixed.pdb")
view.show_mdtraj(base)

NGLWidget()

In [27]:
base.topology

<mdtraj.Topology with 1 chains, 92 residues, 762 atoms, 769 bonds at 0x160e9c820>

In [11]:
from OpenSMOG import OpenSMOG

In [13]:
pdb = OpenSMOG.pdbfile.PDBFile("../data/TrialSystem1/TrialSystem1.pdb")
gro = OpenSMOG.GromacsGroFile("../data/TrialSystem1/TrialSystem1.gro")
top = OpenSMOG.GromacsTopFile("../data/TrialSystem1/TrialSystem1.top")

In [14]:
top

In [3]:
from OpenSMOG import SBM
from pathlib import Path

    ****************************************************************************************    
      **** *** *** *** *** *** *** *** OpenSMOG (v1.1.1) *** *** *** *** *** *** *** ****       

           The OpenSMOG class is used to perform molecular dynamics simulations using           
                     Structure-Based Models (SBM) for biomolecular systems,                     
             and it allows for the simulation of a wide variety of potential forms.             
                      OpenSMOG uses force field files generated by SMOG 2.                      
                             OpenSMOG documentation is available at                             
                  https://opensmog.readthedocs.io and https://smog-server.org                   

                    OpenSMOG is described in: Oliveira and Contessoto et al,                    
              SMOG 2 and OpenSMOG: Extending the limits of structure-based models.              
                    Protein 

In [19]:
sbm_CA = SBM(name='2ci2', time_step=0.0005, collision_rate=1.0, r_cutoff=0.65, temperature=0.5)

In [24]:
sbm_CA.setup_openmm(platform='cpu')

In [42]:
sbm_CA

In [28]:
sbm_CA.saveFolder('../data/test2')

In [5]:
sbm_grofile = "../data/base/base.gro"
sbm_topfile = "../data/base/base.top"
# sbm_xmlfile = "../data/TrialSystem1/TrialSystem1.xml"
# assert Path(sbm_grofile).exists()


In [5]:
import openmm as mm
from openmm import app

In [6]:
top = app.GromacsTopFile(sbm_topfile)
pos = app.GromacsGroFile(sbm_grofile)

In [7]:
dir(top)

['_MoleculeType',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_angleTypes',
 '_atomTypes',
 '_bondTypes',
 '_cmapTypes',
 '_currentCategory',
 '_currentMoleculeType',
 '_defaults',
 '_defines',
 '_dihedralTypes',
 '_elseStack',
 '_genpairs',
 '_ifStack',
 '_implicitTypes',
 '_includeDirs',
 '_moleculeTypes',
 '_molecules',
 '_nonbondTypes',
 '_pairTypes',
 '_processAngle',
 '_processAngleType',
 '_processAtom',
 '_processAtomType',
 '_processBond',
 '_processBondType',
 '_processCmap',
 '_processCmapType',
 '_processConstraint',
 '_processDefaults',
 '_processDihedral',
 '_processDihedralType',
 '_processExclusion',
 '_processFile',
 '_processImplicitType',
 '_pro

In [49]:
dir(mm.openmm.CustomNonbondedForce)

['CutoffNonPeriodic',
 'CutoffPeriodic',
 'NoCutoff',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'addComputedValue',
 'addEnergyParameterDerivative',
 'addExclusion',
 'addFunction',
 'addGlobalParameter',
 'addInteractionGroup',
 'addParticle',
 'addPerParticleParameter',
 'addTabulatedFunction',
 'createExclusionsFromBonds',
 'getComputedValueParameters',
 'getCutoffDistance',
 'getEnergyFunction',
 'getEnergyParameterDerivativeName',
 'getExclusionParticles',
 'getForceGroup',
 'getFunctionParameters',
 'getGlobalParameterDefaultValue',
 'getGlobalParameterName',
 'getInteracti

In [38]:
[[bond.atom1.index, bond.atom2.index] for bond in top.topology.bonds()]

[[0, 1],
 [1, 2],
 [2, 3],
 [3, 4],
 [4, 5],
 [5, 6],
 [6, 7],
 [7, 8],
 [8, 9],
 [9, 10],
 [10, 11],
 [11, 12],
 [12, 13],
 [13, 14],
 [14, 15],
 [15, 16],
 [16, 17],
 [17, 18],
 [18, 19],
 [19, 20],
 [20, 21],
 [21, 22],
 [22, 23],
 [23, 24],
 [24, 25],
 [25, 26],
 [26, 27],
 [27, 28],
 [28, 29],
 [29, 30],
 [30, 31],
 [31, 32],
 [32, 33],
 [33, 34],
 [34, 35],
 [35, 36],
 [36, 37],
 [37, 38],
 [38, 39],
 [39, 40],
 [40, 41],
 [41, 42],
 [42, 43],
 [43, 44],
 [44, 45],
 [45, 46],
 [46, 47],
 [47, 48],
 [48, 49],
 [49, 50],
 [50, 51],
 [51, 52],
 [52, 53],
 [53, 54],
 [54, 55],
 [55, 56],
 [56, 57],
 [57, 58],
 [58, 59],
 [59, 60],
 [60, 61],
 [61, 62],
 [62, 63],
 [63, 64],
 [64, 65],
 [65, 66],
 [66, 67],
 [67, 68],
 [68, 69],
 [69, 70],
 [70, 71],
 [71, 72],
 [72, 73],
 [73, 74],
 [74, 75],
 [75, 76],
 [76, 77],
 [77, 78],
 [78, 79],
 [79, 80],
 [80, 81],
 [81, 82],
 [82, 83],
 [83, 84],
 [84, 85],
 [85, 86],
 [86, 87],
 [87, 88],
 [88, 89],
 [89, 90],
 [90, 91]]

In [37]:
dir(list(top.topology.bonds())[0].atom1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'element',
 'id',
 'index',
 'name',
 'residue']

In [35]:
pos.getPeriodicBoxVectors()

Quantity(value=(Vec3(x=6.354000091552734, y=0, z=0), Vec3(x=0, y=4.506999969482422, z=0), Vec3(x=0, y=0, z=7.470999908447266)), unit=nanometer)

In [67]:
system = top.createSystem(
#     nonbondedMethod="ff.CutoffPeriodic",
    nonbondedCutoff=0.65*mm.openmm.unit.nanometer,
    removeCMMotion=True
)

system.setDefaultPeriodicBoxVectors(*pos.getPeriodicBoxVectors())

nonbonded_force = mm.openmm.CustomNonbondedForce("(sigma/r)^6")
nonbonded_force.addGlobalParameter('sigma', 0.2)

nonbonded_force.setCutoffDistance(0.65*mm.openmm.unit.nanometer)


for i in range(top.topology.getNumAtoms()):
    nonbonded_force.addParticle()

nonbonded_force.createExclusionsFromBonds(
    [[bond.atom1.index, bond.atom2.index] for bond in top.topology.bonds()],
    bondCutoff=1
)

system.addForce(nonbonded_force)
dir(system)

# help(top.loadBondDefinitions)

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'addConstraint',
 'addForce',
 'addParticle',
 'getConstraintParameters',
 'getDefaultPeriodicBoxVectors',
 'getForce',
 'getForces',
 'getNumConstraints',
 'getNumForces',
 'getNumParticles',
 'getParticleMass',
 'getVirtualSite',
 'isVirtualSite',
 'removeConstraint',
 'removeForce',
 'setConstraintParameters',
 'setDefaultPeriodicBoxVectors',
 'setParticleMass',
 'setVirtualSite',
 'this',
 'thisown',
 'usesPeriodicBoundaryConditions']

In [54]:
# dir(system)
# help(system.setConstraintParameters)
# help(system)
# system.getDefaultPeriodicBoxVectors()
# dir(system.getForces()[0])
# help(top.createSystem)

In [60]:
integrator = mm.LangevinIntegrator(300*mm.unit.kelvin, 1/mm.unit.picosecond, 0.002*mm.unit.picoseconds)
platform = mm.Platform.getPlatformByName("CPU")
simulation = app.Simulation(top.topology, system, integrator, platform)
simulation.context.setPositions(pos.positions)


OpenMMException: All Forces must have identical exclusions

In [16]:
help(top.createSystem)

Help on method createSystem in module openmm.app.gromacstopfile:

createSystem(nonbondedMethod=NoCutoff, nonbondedCutoff=Quantity(value=1.0, unit=nanometer), constraints=None, rigidWater=True, implicitSolvent=None, soluteDielectric=1.0, solventDielectric=78.5, ewaldErrorTolerance=0.0005, removeCMMotion=True, hydrogenMass=None, switchDistance=None) method of openmm.app.gromacstopfile.GromacsTopFile instance
    Construct an OpenMM System representing the topology described by this
    top file.
    
    Parameters
    ----------
    nonbondedMethod : object=NoCutoff
        The method to use for nonbonded interactions.  Allowed values are
        NoCutoff, CutoffNonPeriodic, CutoffPeriodic, Ewald, PME, or LJPME.
    nonbondedCutoff : distance=1*nanometer
        The cutoff distance to use for nonbonded interactions
    constraints : object=None
        Specifies which bonds and angles should be implemented with
        constraints. Allowed values are None, HBonds, AllBonds, or HAngles.


In [75]:
list(top.bonds())[1]

Bond(<Atom 1 (CA) of chain 0 residue 1 (PRO)>, <Atom 2 (CA) of chain 0 residue 2 (LEU)>)